<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

---
Приветствую! Меня зовут Антон, и я совсем не против общения на "ты"! Вероятно, многие пошли путем преодобратки данных через TF-IDF, я в данной работе решил попробовать готовый фреймворк Spacy, воодушевившись статьей на https://habr.com/ru/post/504680/.
Работа не изобилует красивым кодом, но на мой взгляд она получилась лаконичной и с неплохим результатом по скорингу, при этом в целях экономии времени на создание векторов и обучение, я урезал исходный датасет. Если работать со всеми данными, полагаю, результат был бы еще лучше.
Заранее хочу поблагодарить за проверку работы и советы :)


## Подготовка

In [3]:
import spacy
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split
import xgboost
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [2]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 45.7 MB 48 kB/s  eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [3]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [4]:
df.sample(5)

,text,toxic
125314,Call it being pushed into a corner or call it ...,1
1618,"And, many taxonomists classify T.durum as a su...",0
115640,Air and Kilometers\nI guess only 'classic song...,0
70199,My bad \n\nEverything on here is so confusing >_<,0
156061,Urartian-Armenians \n\nHey can you do me a fav...,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [6]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Видим несбалансированность целевых классов.

Посмотрим на пример текста:

In [7]:
print(df.text[156205])

"

 Name one time where I have ignored them. Show me right now, give me a link. I have replied to all of your accusations, and I have countered all of your false claims effectively. You are seriously embarrassing yourself. Do you seriously expect me to believe that some random individual is denying all of my claims backed up by evidence. Then this same individual tells you that instead of relying on facts, you have to get more people to agree with you on the talk page, which I did. Then when you ask this same individual why there are have been no changes to the page since they have managed to get more people to agree with them, and the individual replies with the statement of ""the consensus is biased as the proposal."" Now do you seriously want me to believe that after all of that you are not either

1. Biased/Prejudice
2. Taking Orders from another person
3. Being Paid to do so

If you seriously want me to believe that you do not apply to any of those categories and you are just a re

 Далее мы урежем данные, сбалансировав попутно классы по наименьшему классу в соотношении 1:1. Таким образом мы получаем баланс классов и уменьшим время, необходимое для векторизации текстов и обучение моделей.

In [8]:
df_0 = df.query("toxic==0")
df_0.shape

(143346, 2)

In [9]:
df_1 = df.query("toxic==1")
df_1.shape

(16225, 2)

In [10]:
df_0 = df_0.sample(df_1.shape[0])
df_0.shape

(16225, 2)

In [11]:
df_0.head()

,text,toxic
7800,The picture is not showing up?,0
111034,This information is incorrect.,0
116831,Matthew Gagnon is a hoax \n\nIt's a vanity pag...,0
76023,Contested deletion \n\nThis page should not be...,0
48108,""":Please sign your name next time. By the way ...",0


In [12]:
df_r = pd.concat([df_0, df_1], ignore_index=True)

Ниже исправлю логику разбиения и даунсемплинга

Ниже ноутбук носит характер неудачной работы. После изменения логики разбиения выборок, модели стали работать значительно хуже и не дотягивают то требуемого результата. Я пробовал обучать как на сбалансированной выборке так и несбалансированной, результат плохой.

In [13]:
df_r.toxic.value_counts()

0    16225
1    16225
Name: toxic, dtype: int64

In [14]:
df_r.head(4)

,text,toxic
0,The picture is not showing up?,0
1,This information is incorrect.,0
2,Matthew Gagnon is a hoax \n\nIt's a vanity pag...,0
3,Contested deletion \n\nThis page should not be...,0


## Обучение

In [9]:
nlp = spacy.load('ru_core_news_md')

In [16]:
nlp('I love Yandex!').vector.shape

(300,)

In [17]:
# %%time
# df_r['text2vec'] = df_r['text'].apply(lambda x: nlp(x).vector) #векторизуем текст. Операция небыстрая.

In [18]:
# X = df_r['text2vec']
# y = df_r['toxic']

In [19]:
# X_train, X_test, y_train, y_test = train_test_split(
#     X,
#     y,
#     test_size=0.25,
#     stratify=df_r['toxic'],
#     random_state=123,
# )

Воспользуемся функцией очистки текста. В фунцию мы добавим проверку на пустые значения.

In [20]:
stopwords = nlp.Defaults.stop_words

def remove_stop_words(text):
    lst = []
    #text = re.sub(r"[^'a-zA-Z ]", ' ', text) 
    for token in text.split():
        if token.lower() not in stopwords:    #checking whether the word is not 
            lst.append(token)                    #present in the stopword list.
    if ' '.join(lst).strip() == '':
        return np.nan
    return ' '.join(lst).strip()

In [21]:
df['text'] = df['text'].apply(lambda x: remove_stop_words(x))

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159565 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [23]:
df = df.dropna()

In [24]:
df.head()

,text,toxic
0,Explanation edits username Hardcore Metallica ...,0
1,D'aww! matches background colour I'm seemingly...,0
2,"Hey man, I'm trying edit war. It's guy constan...",0
3,""" can't real suggestions improvement - wondere...",0
4,"You, sir, hero. chance remember page that's on?",0


In [25]:
X = df['text']
y = df['toxic']

Делаем предварительное разбиение выборки. Ниже ее придется немного доработать.

In [26]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    #train_size=0.3,
    test_size=0.05,
    stratify=df['toxic'],
    random_state=123,
)

In [27]:
X_test.shape

(7979,)

In [28]:
X_train.shape

(151586,)

In [29]:
y_train.value_counts()

0    136172
1     15414
Name: toxic, dtype: int64

In [30]:
train = pd.concat([X_train, y_train], axis=1)
train['toxic'].value_counts()

0    136172
1     15414
Name: toxic, dtype: int64

In [31]:
train_1 = train.query("toxic==1")
train_0 = train.query('toxic==0').sample(train_1.shape[0])
train = pd.concat([train_0, train_1])
train = train.sample(frac=1).reset_index(drop=True) #перемешаем данные
X_train = train['text']
y_train = train['toxic']

In [32]:
%%time
X_train = X_train.apply(lambda x: nlp(x).vector)

CPU times: user 6min 20s, sys: 1.07 s, total: 6min 21s
Wall time: 6min 21s


In [33]:
%%time
X_test = X_test.apply(lambda x: nlp(x).vector)

CPU times: user 1min 34s, sys: 132 ms, total: 1min 34s
Wall time: 1min 34s


<b> Обучим модель XGBoost:

In [34]:
%%time
xgb_model = xgboost.XGBClassifier()
xgb_model.fit(np.vstack(X_train), y_train) # подсмотрено на https://stackoverflow.com/questions/66092161/xgboost-valueerror-please-reshape-the-input-data-x-into-2-dimensional-matrix-in, иначе возникает ошибка
#predictions = xgb.predict(np.vstack(val['doc_vector']))


/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:41:48] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
CPU times: user 7min 32s, sys: 2.01 s, total: 7min 34s
Wall time: 7min 35s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [35]:
%%time
test_f1_score = f1_score(y_test, xgb_model.predict(np.vstack(X_test)))

CPU times: user 132 ms, sys: 0 ns, total: 132 ms
Wall time: 107 ms


<b> Результирующий скоринг XGBoost: 

In [36]:
test_f1_score

0.636842105263158

<b>Обучим модель Logistic Regression:

In [37]:
%%time
lr_model = LogisticRegression(max_iter=1000, random_state=123)
lr_model.fit(np.vstack(X_train), y_train)

CPU times: user 8.63 s, sys: 7.02 s, total: 15.6 s
Wall time: 15.7 s


LogisticRegression(max_iter=1000, random_state=123)

In [38]:
%%time
test_f1_score = f1_score(y_test, lr_model.predict(np.vstack(X_test)))

CPU times: user 50.6 ms, sys: 65.5 ms, total: 116 ms
Wall time: 112 ms


<b> Результирующий скоринг LogisticRegression: 

In [39]:
test_f1_score

0.6498161764705882

<b> Итог: скоринги низкие, переходим к тестированию модели на основе векторизации TF-IDF

 <h1>Часть 2 (векторизация TF-IDF)<font color='blue'>  </font>

In [1]:
# Импортируем недостающие библиотеки
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
# Заново прочитаем файл и инициализируем наборы для моделей
df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [10]:
# Интереса ради объеденим два набора стоп-слов
stopwords_nlp = nlp.Defaults.stop_words
nltk.download('stopwords')
stopwords_nltk = set(nltk_stopwords.words('english'))
stopwords = stopwords_nlp.union(stopwords_nltk)
len(stopwords)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


382

Фукнция, которая будет очищать, токинизировать и лемматизировать текст.

In [11]:
m = WordNetLemmatizer() 

def prepare_text(text):
    text = (re.sub(r'[^a-zA-Z]', ' ', text)).split(' ')
    text = ' '.join([word for word in text if word not in stopwords])
    text = ' '.join([m.lemmatize(word) for word in text.split()])
    if text == '': # добавляем проверку на пустые строки
        return np.nan
    return text   

In [15]:
%%time
df['prep_text'] =prepare_text df['text'].apply(lambda x: prepare_text(x))

CPU times: user 20.6 s, sys: 22.7 ms, total: 20.6 s
Wall time: 20.7 s


In [16]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   text       159571 non-null  object
 1   toxic      159571 non-null  int64 
 2   prep_text  159535 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.7+ MB


Видим, что после обработки текста, появились nan'ы, удалим их

In [17]:
df = df.dropna()

In [18]:
X = df['prep_text']
y = df['toxic']

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.20,
    stratify=df['toxic'],
    random_state=123,
)

In [19]:
tf_idf = TfidfVectorizer() #stop_words=stopwords)

In [20]:
%%time
X_train = tf_idf.fit_transform(X_train)
X_test = tf_idf.transform(X_test)

CPU times: user 4.29 s, sys: 65.3 ms, total: 4.36 s
Wall time: 4.36 s


<b>Обучим модель Logistic Regression c поиском гиперпараметра 'C' по сетке:

In [50]:
%%time

clf_lr = LogisticRegression(random_state=12345,  class_weight='balanced', max_iter=1000)
parametrs_lr = {'C': np.arange(4.7, 5.3, 0.1)}
grid_lr = GridSearchCV(clf_lr, parametrs_lr, cv=3, scoring='f1')
grid_lr.fit(X_train, y_train)
grid_lr.best_params_

CPU times: user 5min 58s, sys: 12min 9s, total: 18min 7s
Wall time: 18min 8s


{'C': 4.7}

In [25]:
%%time
test_f1_score = f1_score(y_test, grid_lr.predict(X_test))

CPU times: user 20 ms, sys: 8.77 ms, total: 28.8 ms
Wall time: 92.9 ms


In [26]:
test_f1_score

0.7732730611668315

In [53]:
%%time
xgb_model = xgboost.XGBClassifier()
xgb_model.fit(X_train, y_train) 

/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:08:21] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
CPU times: user 4min 27s, sys: 0 ns, total: 4min 27s
Wall time: 4min 28s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [54]:
%%time
test_f1_score = f1_score(y_test, xgb_model.predict(X_test))

CPU times: user 285 ms, sys: 0 ns, total: 285 ms
Wall time: 222 ms


In [55]:
test_f1_score

0.7359467455621302

In [56]:
# обучается очень долго
# estimator = xgboost.XGBClassifier(
#     seed=42
# )

# parameters = {
#     'max_depth': [10],
#     'n_estimators': [10, 100],
#     'learning_rate': [0.2, 0,7]
# }

# grid_xgboost = GridSearchCV(
#     estimator=estimator,
#     param_grid=parameters,
#     scoring='f1',
#     cv=3,
#     verbose=True
# )

In [57]:
# %%time
# grid_xgboost.fit(X_train, y_train)

In [58]:
# grid_xgboost.best_params_

In [59]:
# test_f1_score = f1_score(y_test, grid_xgboost.predict(X_test))

In [60]:
# test_f1_score

<b>
Вывод: в результате работы было выяснено, что модели, работающие на предобработанных такстах с помощью векторизация библиотекой Spacy не дает требуемой точности обучаемых моделей. Поэтому было использовано простое решение с использованием векторизации на основые ТF-IDF.
Оптимальной моделью с требуемой точностью можно назвать логистическую регрессию, в результате подбора гиперпараметров с помощью GridSearch, которое заняло около 6 минут удалось получить точность метрики F1 на тестовом наборе равным 0.773, что выше требуемого в задаче порога.
    